## Current Python Version

Ensure the python version running in the jupyter kernel is the same as expected

In [1]:
# from platform import python_version

# print(python_version())


List installed packages, should match the ones in the pipfile (and their dependencies)

In [2]:
# pip list

## Internal Module Imports
Code required to enable project's internally defined modulos into the jupyter notebook environment

In [3]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
# from electricity import ElectricalSystem, Load, Generator
from dto import (
    COST_PRESETS, 
    ElectricalSystemSpecs, 
    EpsilonSpecs,
    GeneratorSpecs, 
    LoadSpecs, 
    NodeStatePower, 
    SystemHistory, 
    )
    
# from learning import ModelTrainer, ModelTester, LearningParams
from learning import LearningParams
from learning.cost import CostModelTrainer, CostModelTester

/home/fmello/.local/share/virtualenvs/2019-MSc_Thesis-4pUnri4s/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/fmello/.local/share/virtualenvs/2019-MSc_Thesis-4pUnri4s/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/fmello/.local/share/virtualenvs/2019-MSc_Thesis-4pUnri4s/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 

ImportError: cannot import name 'CostModelTrainer' from 'learning.cost' (/mnt/c/Users/flavi/Documents/Study/City-UniLondon/Thesis/2019-MSc_Thesis/app/learning/cost/__init__.py)

Determine the experiment params

In [ ]:
from dto import EpsilonSpecs

LearningParams(
    gamma=0.9,   # Gamma (Discount)
    tau=0.001, # Tau
    epsilonSpecs = EpsilonSpecs( # Epsilon explore/exploit control
        thresholdProgress = 0.6, # % of steps where decay change should happen
        thresholdValue = 0.5, # Value at which decay change would happen
        finalValue = 0.0001, # Value at the end of the experiment
    ),
    numEpisodes=2000, # Number of learning episodes to run
    maxSteps=100,   # Number of steps per learning episode
    bufferSize=500, # Experience Buffer Size
    traceLength=15,     # Number of steps each sampled episode should contain
    batchSize=40,     # Number of episodes sampled from experience buffer
    updateInterval=4, # Run update cycle every N steps
    electricalSystemSpecs = ElectricalSystemSpecs(
        loads=[LoadSpecs(id_="L1", basePower=3.15, noiseLevel=0)],
        generators=[
            GeneratorSpecs(id_="G1", basePower=2, costProfile=COST_PRESETS.COAL_OLD, minPower=0.5, maxPower=5.0, noiseLevel=0.1),
            GeneratorSpecs(id_="G2", basePower=2, costProfile=COST_PRESETS.OIL_OLD, minPower=0.5, maxPower=5.0, noiseLevel=0.1),
        ],
    ),
    modelName='2Gen-OriginalTest-C65-2k'
)
# outputDifferential = (totalOutputDestination - totalOutputOrigin)
# costDifferential = elecSystem.getCostOptimalDiferential()
# earnedReward = 2**(10-abs(costDifferential) - abs(outputDifferential))# TODO Calculate reward according to a given strategytrategy
# allRewards.append(earnedReward)
  

Train Agents

In [ ]:
# allAgents = ModelTrainer.trainAgents()
import math
def rewardFn(totalOutputOrigin, totalOutputDestination, costDifferential, totalCost):
    totalCost = totalCost/(10000*100.0)
    outputDifferential = (totalOutputDestination - totalOutputOrigin)/totalOutputOrigin
#     outputComponent = 1.0/math.cosh(5*outputDifferential) # 1/cosh(x) = sech(x) which peaks at x=0 and approaches 0 as x= +- inf
#     costComponent = 1.0/math.cosh(costDifferential)
#     earnedReward = 2**(10*(costComponent+outputComponent)/2)
    earnedReward = 2**(-1*((totalCost**2)*200 + (outputDifferential**2)*500))
    return earnedReward

allAgents = CostModelTrainer.trainAgents(rewardFn)

In [ ]:
STEPS_TO_TEST = 300

# testSpecs = ElectricalSystemSpecs(
#         loads=[LoadSpecs(id_="L1", basePower=3.0, noiseLevel=0.1)],
#         generators=[
#             GeneratorSpecs(id_="G1", basePower=1.5, costProfile=COST_PRESETS.COAL, minPower=0.5, maxPower=2.0, noiseLevel=0.0),
#             GeneratorSpecs(id_="G2", basePower=1.5, costProfile=COST_PRESETS.OIL, minPower=0.5, maxPower=2.0, noiseLevel=0.0),
#         ],
# )
# electricalSystem, rewards = ModelTester.testAgents(testSpecs, LearningParams().modelName, STEPS_TO_TEST)


# electricalSystem, rewards = ModelTester.testAgents(LearningParams().electricalSystemSpecs, LearningParams().modelName, STEPS_TO_TEST)
electricalSystem, rewards = CostModelTester.testAgents(LearningParams().electricalSystemSpecs, LearningParams().modelName, rewardFn, STEPS_TO_TEST)

Alternatively, one can plot the data to better inspect the progression

In [ ]:
import matplotlib.pyplot as plt
from plots import plotAll

Plot the reward progression of the model testing

In [ ]:
steps = range(STEPS_TO_TEST)
plt.scatter(steps, rewards)
sum(rewards)

Plot all graphs related to the electrical system state during the model testing

In [ ]:
# plotAll(electricalSystem.systemHistory)

In [ ]:
# # Show the last episode ran during training
from learning.learning_state import LearningState
plotAll(LearningState().episode.electricalSystem.systemHistory)

In [ ]:
for rewList in LearningState().model.allRewards:
    steps = range(len(rewList))
    plt.scatter(steps, rewList)
    
cummRew = LearningState().model.cummRewardList
steps = range(len(cummRew))
plt.scatter(steps, cummRew)
# len(LearningState().model.cummRewardList)

In [ ]:
from learning.learning_state import LearningState
import numpy as np

rewList = LearningState().model.cummRewardList

In [ ]:
allRewards = LearningState().model.allRewards


for r in allRewards[0::100]:
    plt.figure()
    plt.scatter(range(len(r)), r)
# sum(rewards)

In [ ]:
plt.scatter(range(len(rewList)), rewList)

In [ ]:
from plots.plot_constants import COLOR_PALETTE, FONT_SIZES

history = electricalSystem.systemHistory
# Get series to be plotted
stepsSeries = history.steps
loads = history.loads
totalLoadSeries = history.totalLoad
generators = history.generators
totalPowerSeries = history.totalPower

plt.figure(0)

# Declare colors to be used
colorTotalLoad = COLOR_PALETTE[0]
colorTotalPower = COLOR_PALETTE[1]
colorsIndividualNodes = COLOR_PALETTE[3:]

# Plot total power/load data
plt.plot(stepsSeries, totalLoadSeries, color=colorTotalLoad)
plt.plot(stepsSeries, totalPowerSeries, color=colorTotalPower)
legendFields = ['Total Load', 'Total Power']

# Multiple scenarios involve a single load, so we make it optional to plot that single load
if(False):
    for idx, loadId in enumerate(loads):
        # Since num loads is variable, colors may wrap around the palette
        loadColor = colorsIndividualNodes[idx % len(colorsIndividualNodes)]
        loadLegend = loadId
        loadSeries = loads[loadId]
        plt.plot(stepsSeries, loadSeries, color=loadColor, linestyle='--')
        legendFields.append(loadLegend)

for idx, generatorId in enumerate(generators):
    # Since num generators is variable, colors may wrap around the palette
    generatorColor = colorsIndividualNodes[idx % len(colorsIndividualNodes)]
    generatorLegend = generatorId
    generatorSeries = generators[generatorId]
    plt.plot(stepsSeries, generatorSeries, color=generatorColor)
    legendFields.append(generatorLegend)
    

summedPow = [x + y for x, y in zip(generators.get('G1'), generators.get('G2'))]
plt.plot(stepsSeries, summedPow, color=COLOR_PALETTE[2])
legendFields.append("Summed Pow")


plt.legend(legendFields)
plt.xlabel('Steps', fontsize=FONT_SIZES['AXIS_LABEL'])
plt.ylabel('Power (pu)', fontsize=FONT_SIZES['AXIS_LABEL'])

plt.title('System Power (pu) x Time (Steps)', fontsize=FONT_SIZES['TITLE'])

plt.show()

In [ ]:
rewardFn(0.1, 0.2, 0.3, 0.4)